In [ ]:
import networkx as nx
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import roc_auc_score, precision_score, roc_curve, precision_recall_curve, auc